<a href="https://colab.research.google.com/github/Rob-A-B/wildfire-hotspots/blob/main/queimadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import re

In [ ]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

hoje = datetime.date.today()
BASE   = "/content/projeto_pcb"
BRONZE_RAW = f"{BASE}/dados/bronze/raw"                   # onde guardamos os arquivos "brutos" por partição
BRONZE_CUR = f"{BASE}/dados/bronze/current"               # dataset consolidado (único arquivo)
os.makedirs(BRONZE_RAW, exist_ok=True)
os.makedirs(BRONZE_CUR, exist_ok=True)

# Define the path to your CSV files in Google Drive
# **IMPORTANT:** Replace 'My Drive/Your_Folder_With_CSVs' with the actual path to your folder
DRIVE_CSV_PATH = "/content/drive/My Drive/Your_Folder_With_CSVs"
os.makedirs(DRIVE_CSV_PATH, exist_ok=True) # Create the directory if it doesn't exist


# 1) Descobrir todos os CSVs diretamente em /content/drive/My Drive/Your_Folder_With_CSVs
arquivos = [f for f in os.listdir(DRIVE_CSV_PATH) if f.endswith(".csv") and os.path.isfile(f"{DRIVE_CSV_PATH}/{f}")]
print(f"📦 Encontrados {len(arquivos)} CSVs em {DRIVE_CSV_PATH}.\n")

# 2) Mover/copiar para Bronze/raw particionado e montar lista de DataFrames
dfs = []
for nome in arquivos:
    caminho_src = f"{DRIVE_CSV_PATH}/{nome}"

    # extrai AAAAMM do nome (ex.: focos_mensal_br_202311.csv)
    m = re.search(r'(\d{6})', nome)
    aaaamm = m.group(1) if m else f"{hoje.year}{hoje:%m}"
    ano, mes = aaaamm[:4], aaaamm[4:]

    # destino particionado
    dest_dir = os.path.join(BRONZE_RAW, f"ano={ano}", f"mes={mes}")
    os.makedirs(dest_dir, exist_ok=True)
    caminho_dst = os.path.join(dest_dir, nome)

    # Copy (to keep the original in Drive)
    shutil.copy(caminho_src, caminho_dst)
    print(f"✅ Copiado: {nome}  ➜  {caminho_dst}")

    # lê e adiciona metadados de partição
    try:
        df_temp = pd.read_csv(caminho_dst)
        df_temp["ano"] = int(ano)
        df_temp["mes"] = int(mes)
        df_temp["origem_arquivo"] = nome
        dfs.append(df_temp)
    except Exception as e:
        print(f"⚠️ Erro ao ler {caminho_dst}: {e}")


# 3) Consolida tudo num único DataFrame (Bronze/current)
if dfs:
    bronze_df = pd.concat(dfs, ignore_index=True)

    # Idempotência simples: remove duplicatas exatas de linhas
    # (Se quiser, troque por subset=["id","datahora","latitude","longitude"] se existirem)
    bronze_df = bronze_df.drop_duplicates()

    print(f"\n🧱 Bronze consolidado: {len(bronze_df):,} linhas, {bronze_df.shape[1]} colunas")
    display(bronze_df.head())
    display(bronze_df.tail())

    # 4) Salva consolidado (CSV + Parquet)
    out_csv = os.path.join(BRONZE_CUR, "focos_bronze_consolidado.csv")
    out_parquet = os.path.join(BRONZE_CUR, "focos_bronze_consolidado.parquet")

    bronze_df.to_csv(out_csv, index=False)
    try:
        bronze_df.to_parquet(out_parquet, index=False)
    except Exception as e:
        print(f"ℹ️ Parquet não salvo (instale pyarrow/fastparquet se quiser): {e}")

    print(f"\n💾 Consolidado salvo em:\n - {out_csv}\n - {out_parquet} (se disponível)")
else:
    print("⚠️ Nada para consolidar na Bronze.")

In [ ]:
# Caminho da camada Bronze (já criada antes)
hoje = datetime.date.today()
BASE = "/content/projeto_pcb"
BRONZE = f"{BASE}/dados/bronze/ano={hoje.year}/mes={hoje:%m}"

# Lista todos os CSVs na raiz do Colab
arquivos = [f for f in os.listdir("/content/") if f.endswith(".csv")]

if not arquivos:
    print("⚠️ Nenhum arquivo CSV encontrado.")
else:
    print(f"📦 {len(arquivos)} arquivos encontrados.\n")

    for nome_arquivo in arquivos:
        origem = f"/content/{nome_arquivo}"
        destino = os.path.join(
            BRONZE,
            f"{os.path.splitext(nome_arquivo)[0]}_{hoje}.csv"
        )

        # Move o arquivo
        shutil.copy(origem, destino)
        print(f"✅ Arquivo movido para Bronze: {destino}")

        # Pré-visualiza o conteúdo
        df = pd.read_csv(destino)
        print(f"🧾 {nome_arquivo}: {len(df)} linhas")
        display(df.head())

# up = files.upload()
# nome_arquivo = list(up.keys())[0]
# dest = os.path.join(BRONZE, f"{os.path.splitext(nome_arquivo)[0]}_{datetime.date.today()}.csv")
# shutil.move('focos_mensal_br_202301', dest)
# print("Salvo em bronze:", dest)

⚠️ Nenhum arquivo CSV encontrado.


In [ ]:
bronze_df.head()

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,...,ano,mes,origem_arquivo,4,33,6,0.1,0.72,Amazônia,0.8
0,80c0c8a9-2e6a-3e6a-906d-43417d9e59a8,2.1767,-61.8534,2025-03-01 00:00:00,GOES-16,IRACEMA,RORAIMA,Brasil,1400282.0,14.0,...,2025,3,focos_mensal_br_202503.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,94d07fa8-0b51-344c-a962-74a491340983,-9.2590,-60.3090,2025-03-01 00:00:00,GOES-16,COLNIZA,MATO GROSSO,Brasil,5103254.0,51.0,...,2025,3,focos_mensal_br_202503.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,080aab7f-cba3-359b-840c-1e33ec757337,0.9569,-60.5233,2025-03-01 00:00:00,GOES-16,RORAINÓPOLIS,RORAIMA,Brasil,1400472.0,14.0,...,2025,3,focos_mensal_br_202503.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0981cb4b-3ab6-333a-9071-473a8dc8f717,2.1949,-61.8532,2025-03-01 00:00:00,GOES-16,IRACEMA,RORAIMA,Brasil,1400282.0,14.0,...,2025,3,focos_mensal_br_202503.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,034760e3-518e-3b08-b96f-030c998f34df,2.1767,-61.8534,2025-03-01 00:10:00,GOES-16,IRACEMA,RORAIMA,Brasil,1400282.0,14.0,...,2025,3,focos_mensal_br_202503.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
paths = sorted(glob.glob(os.path.join(BRONZE, "*.csv")))
assert paths, "Nenhum CSV encontrado em bronze."
df = pd.concat((pd.read_csv(p) for p in paths), ignore_index=True)

esperadas = {"X","Y","R","G","B"}
assert esperadas.issubset(set(df.columns)), f"Faltam colunas: {esperadas - set(df.columns)}"
if "Grey" not in df.columns:
    df["Grey"] = 0  # se não existir, inicializa


for c in ["X","Y","R","G","B","Grey"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=["X","Y","R","G","B"]).copy()


df["X"] = df["X"].round().astype("Int64")
df["Y"] = df["Y"].round().astype("Int64")
for c in ["R","G","B"]:
    df[c] = df[c].clip(0,255).round().astype(np.uint8)


df["Grey"] = df["Grey"].fillna(0).clip(0,1).astype(int)


df["dt_carga"] = datetime.date.today().isoformat()

# Salva silver
silver_csv = os.path.join(SILVER, f"pcb_xyrgb_silver_{datetime.date.today()}.csv")
df.to_csv(silver_csv, index=False)
print("Silver salvo:", silver_csv, "| linhas:", len(df))



In [ ]:
import os, glob, datetime
import pandas as pd
import numpy as np

paths = sorted(glob.glob(os.path.join(BRONZE, "*.csv")))
assert paths, "Nenhum CSV encontrado em bronze."

#LEMBRAR dessa linha mais na frente quando tiver mais meses / tratar isso no bronze
#df = pd.concat((pd.read_csv(p) for p in paths), ignore_index=True)

esperadas = {"id","lat","lon","data_hora_gmt","satelite","municipio","estado","pais"}
faltando = esperadas - set(df.columns)
assert not faltando, f"Faltam colunas no CSV: {faltando}"

# id como texto (alguns ids têm hífens/UUID)
bronze_df["id"] = bronze_df["id"].astype(str).str.strip()


bronze_df["lat"] = pd.to_numeric(bronze_df["lat"], errors="coerce")
bronze_df["lon"] = pd.to_numeric(bronze_df["lon"], errors="coerce")

bronze_df["data_hora_gmt"] = pd.to_datetime(bronze_df["data_hora_gmt"], errors="coerce", utc=True)

for c in ["satelite","municipio","estado","pais"]:
    bronze_df[c] = bronze_df[c].astype(str).str.strip()

for c in ["municipio_id","estado_id","pais_id"]:
    if c in df.columns:
        bronze_df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

bronze_df = bronze_df.dropna(subset=["lat","lon","data_hora_gmt"]).copy()

#desfazer
bronze_df["lat"] = bronze_df["lat"].round(6)
bronze_df["lon"] = bronze_df["lon"].round(6)

bronze_df["ano"] = df["data_hora_gmt"].bronze_dt.year.astype("Int64")
bronze_df["mes"] = df["data_hora_gmt"].bronze_dt.month.astype("Int64")
bronze_df["dia"] = df["data_hora_gmt"].bronze_dt.day.astype("Int64")

# carga que prof pediu
bronze_df["dt_carga"] = datetime.date.today().isoformat()

# Salva silver
silver_csv = os.path.join(SILVER, f"focos_silver_{datetime.date.today():%Y%m%d}.csv")
os.makedirs(SILVER, exist_ok=True)
df.to_csv(silver_csv, index=False)

print("Silver salvo:", silver_csv, "| linhas:", len(df))
df.head()


NameError: name 'BRONZE' is not defined

In [ ]:
padrao_silver = "focos_silver_*.csv"   # ajuste se usou outro nome

paths = sorted(glob.glob(os.path.join(SILVER, padrao_silver)))
assert paths, f"Nenhum CSV Silver encontrado em: {SILVER}"

# parse_dates  reforçandos a conversão em seguida
df = pd.concat((pd.read_csv(p, parse_dates=["data_hora_gmt"]) for p in paths),
               ignore_index=True)

# Garante que data_hora_gmt é datetime
df["data_hora_gmt"] = pd.to_datetime(df["data_hora_gmt"], errors="coerce", utc=True)

# Normalizações úteis para agregação
df["data"] = df["data_hora_gmt"].dt.date
df["ano"]  = df["data_hora_gmt"].dt.year
df["mes"]  = df["data_hora_gmt"].dt.month
df["hora"] = df["data_hora_gmt"].dt.hour

por_estado_mes = (
    df.groupby(["estado","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)


por_municipio_dia = (
    df.groupby(["estado","municipio","data"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["data","qtd_focos"], ascending=[True,False])
)


por_satelite_mes = (
    df.groupby(["satelite","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)

por_satelite_total = (
    df.groupby("satelite")
      .size()
      .reset_index(name="qtd_focos")
      .assign(perc=lambda x: (x["qtd_focos"] / x["qtd_focos"].sum())*100)
      .sort_values("qtd_focos", ascending=False)
      .round({"perc": 2})
)

df["lat_cell"] = df["lat"].round(2)
df["lon_cell"] = df["lon"].round(2)
grade_espacial = (
    df.groupby(["lat_cell","lon_cell"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values("qtd_focos", ascending=False)
)

# ------------------------------
# Salva em GOLD
# ------------------------------
os.makedirs(GOLD, exist_ok=True)
hoje = datetime.date.today().isoformat()

g1 = os.path.join(GOLD, f"gold_focos_por_estado_mes_{hoje}.csv")
g2 = os.path.join(GOLD, f"gold_focos_por_municipio_dia_{hoje}.csv")
g3 = os.path.join(GOLD, f"gold_focos_por_satelite_mes_{hoje}.csv")
g3b= os.path.join(GOLD, f"gold_focos_por_satelite_total_{hoje}.csv")
g4 = os.path.join(GOLD, f"gold_grade_espacial_{hoje}.csv")

por_estado_mes.to_csv(g1, index=False)
por_municipio_dia.to_csv(g2, index=False)
por_satelite_mes.to_csv(g3, index=False)
por_satelite_total.to_csv(g3b, index=False)
grade_espacial.to_csv(g4, index=False)

print("Gold salvo:")
print(" -", g1)
print(" -", g2)
print(" -", g3)
print(" -", g3b)
print(" -", g4)

try:
    ult_ano  = por_estado_mes["ano"].max()
    ult_mes  = por_estado_mes.query("ano == @ult_ano")["mes"].max()
    top_estados = (
        por_estado_mes
        .query("ano == @ult_ano and mes == @ult_mes")
        .nlargest(10, "qtd_focos")
    )
    print(f"\nTOP estados no mês mais recente ({ult_ano}-{ult_mes:02d}):")
    print(top_estados.to_string(index=False))
except Exception as e:
    print("\n[Aviso] Não foi possível imprimir TOP estados do mês mais recente:", e)

print("\n% por satélite (total):")
print(por_satelite_total.to_string(index=False))

Gold salvo:
 - /content/projeto_pcb/dados/gold/gold_focos_por_estado_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_municipio_dia_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_total_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_grade_espacial_2025-10-09.csv

TOP estados no mês mais recente (2024-03):
            estado  ano  mes  qtd_focos
           RORAIMA 2024    3       1767
       MATO GROSSO 2024    3       1508
             BAHIA 2024    3        936
MATO GROSSO DO SUL 2024    3        562
             GOIÁS 2024    3        429
         SÃO PAULO 2024    3        282
      MINAS GERAIS 2024    3        260
         TOCANTINS 2024    3        132
              PARÁ 2024    3         98
           ALAGOAS 2024    3         93

% por satélite (total):
 satelite  qtd_focos  perc
  NOAA-20       1870 28.18
  NPP-375       1399 21.08
 NPP-375D      

In [ ]:
# cont = df["Grey"].value_counts().sort_index()
# plt.bar(cont.index.astype(str), cont.values)
# plt.title("Distribuição da classe Grey")
# plt.xlabel("Grey (0=não cinza, 1=cinza)")
# # plt.ylabel("n")
# plt.show()


In [ ]:
# agg = df.groupby("Grey")[["R","G","B"]].mean().reset_index()
# for ch in ["R","G","B"]:
#     plt.bar(agg["Grey"].astype(str), agg[ch], label=ch)
#     plt.title(f"Média do canal {ch} por Grey")
#     plt.xlabel("Grey")
#     plt.ylabel(f"Média {ch} (escala {'0–255' if df['R'].max()>1 else '0–1'})")
#     plt.show()
